<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/C_Copy_of_Lab_1_Big_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μεγάλο Dataset (B10 - Epileptic Seizure Recognition)

In [1]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 7.0MB 9.0MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)


# Β. Εισαγωγή του Dataset

Multiclass Classification: Το συγκεκριμένο dataset χρησιμοποιείται και για multiclass και για binary (1 vs 2-5) classification. Λόγω του παραπάνω, και καθώς στο small dataset κάναμε binary classification επιλέξαμε να κάνουμε multiclass σε αυτό.

1. Έγινε καταγραφή της δραστηριότητας του εγκεφάλου 500 ατόμων για 23.6 δευτερόλεπτα/καταγραφή. Στόχος είναι η λήψη σωστής απόφασης περί του αν το άτομο βρίσκεται σε επιληπτική κρίση ή όχι. Έγινε δειγματοληψία κάθε καταγραφής, η οποία οδήγησε σε 4097 δείγματα. Τα 4097 δείγματα χωρίστηκαν σε 23 κομμάτια, με το καθένα να περιέχει 178 σημεία, που αντιστοιχούν σε 1 δευτερόλεπτο καταγραφής. Έτσι δημιουργήθηκαν 23*500 = 11500 γραμμές-δείγματα, καθένα από τα οποία αποτελείται από 178 σημεία που αντιστοιχούν σε 1 δευτερόλεπτο και αποτελόυν τη διάσταση των δεδομένων εισόδου. Η τελευταία στήλη περιέχει τις ετικέτες, οι οποίες παίρνουν τιμές 1-5, με την περίπτωση 1 να αφορά τις περιπτώσεις όπου το άτομο είχε επιληπτική κρίση και τις 2-5 τις περιπτώσεις όπου δεν είχε. 


In [0]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

In [3]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00388/data.csv","ESR.csv")
print("All the files are downloaded")

Download file... ESR.csv ...
File downloaded
All the files are downloaded


In [4]:
!ls

ESR.csv  sample_data


In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("ESR.csv")

2. Όπως είπαμε παραπάνω και όπως φαίνεται παρακάτω, υπάρχουν 11500 δείγματα, το καθένα με 178 χαρακτηριστικά, τα οποία αφορούν τις μεταβολές της ηλεκτρικής τάσης των σημάτων στους νευρώνες του ανθρωπίνου εγκεφάλου. Συνεπώς είναι διατεταγμένα. Επίσης είναι ακέραιοι αριθμοί.
3. Υπάρχουν επικεφαλίδες στην πρώτη γραμμή και στοιχεία για το εκάστοτε δείγμα στην πρώτη στήλη, τα οποία θα πρέπει να αφαιρεθούν.
4. 

In [6]:
print(df.shape)

(11500, 180)


In [7]:
df #print the dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,-38,-10,35,64,113,152,164,127,50,-47,-121,-138,-125,-101,-50,11,39,24,48,64,46,13,-19,-61,-96,-130,-132,-116,-115,-71,...,54,90,111,107,64,32,18,-25,-69,-65,-44,-33,-57,-88,-114,-130,-114,-83,-53,-79,-72,-85,-109,-98,-72,-65,-63,-11,10,8,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,232,237,258,212,2,-267,-605,-850,-1001,-1109,-1090,-967,-746,-464,-152,118,318,427,473,485,447,397,339,312,314,326,335,332,324,310,...,27,146,229,269,297,307,303,305,306,307,280,231,159,85,51,43,62,63,63,69,89,123,136,127,102,95,105,131,163,168,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,-99,-94,-96,-104,-103,-92,-75,-69,-69,-53,-37,-14,-10,-39,-78,-102,-98,-80,-54,-40,-35,-39,-32,-13,7,34,41,33,6,...,-82,-107,-126,-124,-108,-84,-68,-61,-56,-63,-62,-33,1,28,45,37,48,62,80,66,23,-11,-39,-44,-42,-45,-48,-42,-6,29,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,-72,-68,-74,-80,-83,-73,-68,-61,-58,-59,-64,-79,-84,-97,-94,-84,-77,-75,-72,-68,-76,-76,-72,-67,-69,-69,-69,-67,-68,...,-69,-66,-74,-69,-61,-51,-45,-45,-49,-58,-64,-78,-80,-90,-87,-83,-78,-64,-38,-22,-29,-42,-51,-68,-71,-69,-69,-74,-74,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,-90,-103,-84,-43,-9,3,-21,-60,-96,-103,-75,-29,14,55,78,73,28,-13,-43,-68,-78,-75,-55,-41,-19,-20,-29,-36,-20,...,38,26,10,-4,-13,-8,0,10,19,29,57,63,45,7,-13,-23,-9,9,11,3,-1,-2,4,18,27,27,14,15,11,10,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,-57,-64,-73,-79,-76,-70,-63,-57,-57,-50,-45,-34,-33,-32,-30,-24,-24,-18,-9,-2,0,2,-3,-5,-13,-23,-39,-53,-59,...,45,55,60,69,68,58,54,44,40,38,39,39,32,23,22,14,13,15,18,20,20,25,28,29,29,29,26,24,24,20,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,136,78,8,-66,-132,-180,-210,-227,-225,-212,-192,-168,-144,-117,-88,-54,-21,6,29,46,60,76,92,110,128,152,171,150,91,21,...,69,58,38,16,25,75,165,260,343,406,456,471,461,412,319,175,-5,-171,-293,-357,-378,-370,-346,-316,-278,-241,-201,-162,-126,-94,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,14,-1,-10,14,44,77,61,42,32,29,22,26,35,81,66,50,57,91,121,111,73,38,23,35,21,11,-12,2,17,61,...,-90,-62,-38,-40,-21,-23,-11,-4,-9,-22,-42,-49,-48,-40,-40,-46,-43,-67,-67,-50,-18,-14,-39,-74,-86,-75,-68,-57,-78,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,29,22,6,1,-28,-37,-35,-35,-45,-64,-105,-140,-157,-157,-147,-153,-147,-126,-112,-83,-56,-41,-40,-38,-34,-47,-41,-40,-42,-49,...,-98,-77,-60,-73,-88,-97,-118,-108,-100,-97,-91,-109,-122,-134,-137,-107,-95,-67,-54,-40,-31,-19,7,22,46,68,87,97,105,114,121,135,148,143,116,86,68,59,55,3


In [8]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))

We have  180  attributes.
1         Unnamed: 0                              type:     object              
2         X1                                      type:     int64               
3         X2                                      type:     int64               
4         X3                                      type:     int64               
5         X4                                      type:     int64               
6         X5                                      type:     int64               
7         X6                                      type:     int64               
8         X7                                      type:     int64               
9         X8                                      type:     int64               
10        X9                                      type:     int64               
11        X10                                     type:     int64               
12        X11                                     type:     int64               
13

5. Διαγράφουμε την πρώτη κολώνα, ώστε όλες οι κολώνες εκτός της τελευταίας να περιέχουν τιμές των χαρακτηριστικών, με την τελευταία να περιέχει τις ετικέτες.

In [9]:
df = df.drop(df.columns[[0]], axis=1)
df

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,-10,35,64,113,152,164,127,50,-47,-121,-138,-125,-101,-50,11,39,24,48,64,46,13,-19,-61,-96,-130,-132,-116,-115,-71,-14,...,54,90,111,107,64,32,18,-25,-69,-65,-44,-33,-57,-88,-114,-130,-114,-83,-53,-79,-72,-85,-109,-98,-72,-65,-63,-11,10,8,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,386,382,356,331,320,315,307,272,244,232,237,258,212,2,-267,-605,-850,-1001,-1109,-1090,-967,-746,-464,-152,118,318,427,473,485,447,397,339,312,314,326,335,332,324,310,312,...,27,146,229,269,297,307,303,305,306,307,280,231,159,85,51,43,62,63,63,69,89,123,136,127,102,95,105,131,163,168,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,-99,-94,-96,-104,-103,-92,-75,-69,-69,-53,-37,-14,-10,-39,-78,-102,-98,-80,-54,-40,-35,-39,-32,-13,7,34,41,33,6,-15,...,-82,-107,-126,-124,-108,-84,-68,-61,-56,-63,-62,-33,1,28,45,37,48,62,80,66,23,-11,-39,-44,-42,-45,-48,-42,-6,29,57,64,48,19,-12,-30,-35,-35,-36,5
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,-72,-68,-74,-80,-83,-73,-68,-61,-58,-59,-64,-79,-84,-97,-94,-84,-77,-75,-72,-68,-76,-76,-72,-67,-69,-69,-69,-67,-68,-69,...,-69,-66,-74,-69,-61,-51,-45,-45,-49,-58,-64,-78,-80,-90,-87,-83,-78,-64,-38,-22,-29,-42,-51,-68,-71,-69,-69,-74,-74,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,-90,-103,-84,-43,-9,3,-21,-60,-96,-103,-75,-29,14,55,78,73,28,-13,-43,-68,-78,-75,-55,-41,-19,-20,-29,-36,-20,1,...,38,26,10,-4,-13,-8,0,10,19,29,57,63,45,7,-13,-23,-9,9,11,3,-1,-2,4,18,27,27,14,15,11,10,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,-57,-64,-73,-79,-76,-70,-63,-57,-57,-50,-45,-34,-33,-32,-30,-24,-24,-18,-9,-2,0,2,-3,-5,-13,-23,-39,-53,-59,-63,...,45,55,60,69,68,58,54,44,40,38,39,39,32,23,22,14,13,15,18,20,20,25,28,29,29,29,26,24,24,20,15,16,12,5,-1,-18,-37,-47,-48,2
11496,-47,-11,28,77,141,211,246,240,193,136,78,8,-66,-132,-180,-210,-227,-225,-212,-192,-168,-144,-117,-88,-54,-21,6,29,46,60,76,92,110,128,152,171,150,91,21,-29,...,69,58,38,16,25,75,165,260,343,406,456,471,461,412,319,175,-5,-171,-293,-357,-378,-370,-346,-316,-278,-241,-201,-162,-126,-94,-65,-33,-7,14,27,48,77,117,170,1
11497,14,6,-13,-16,10,26,27,-9,4,14,-1,-10,14,44,77,61,42,32,29,22,26,35,81,66,50,57,91,121,111,73,38,23,35,21,11,-12,2,17,61,88,...,-90,-62,-38,-40,-21,-23,-11,-4,-9,-22,-42,-49,-48,-40,-40,-46,-43,-67,-67,-50,-18,-14,-39,-74,-86,-75,-68,-57,-78,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,-40,-25,-9,-12,-2,12,7,19,22,29,22,6,1,-28,-37,-35,-35,-45,-64,-105,-140,-157,-157,-147,-153,-147,-126,-112,-83,-56,-41,-40,-38,-34,-47,-41,-40,-42,-49,-56,...,-98,-77,-60,-73,-88,-97,-118,-108,-100,-97,-91,-109,-122,-134,-137,-107,-95,-67,-54,-40,-31,-19,7,22,46,68,87,97,105,114,121,135,148,143,116,86,68,59,55,3


6. Δεν υπάρχουν απουσιάζουσες τιμές.
7. Χρησιμοποιούμε την bincount για να μετρήσουμε τη συχνότητα των κατηγοριών. Παρατηρούμε οτι έχουμε ενα εξαιρετικά ισορροπημένο dataset. Και οι 5 κατηγορίες είναι ισοπληθείς. 


In [10]:
labels = df.iloc[:, 178]
df = df.iloc[:, 0:178]   #remove lables from set
print("frequencies:", np.bincount(labels)[1:])

frequencies: [2300 2300 2300 2300 2300]


8. Διαχωρίζουμε σε train και test set. Οι τιμές των χαρακτηριστικών αφορούν τα εγκεφαλικά σήματα (τάσεις), άρα είναι διατεταγμένα.

In [11]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.3)
print(len(train))

8050


# Γ. Baseline Classification

##### Dummy Classifiers

Εκπαιδεύουμε τους classifiers με τις default τιμές για να δούμε συγκρίνουμε τα αποτέλεσματα πριν και μετά την προεργασία. <br>

Αρχίζουμε με τους Dummy Classifiers. Παρατηρούμε οτι επιτυγχάνουμε σε όλους περίπου 20% επιτυχία, όπως ήταν αναμενόμενο καθώς έχουμε 5 κατηγορίες.

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dc_uniform = DummyClassifier(strategy="uniform")
dc_constant_1 = DummyClassifier(strategy="constant", constant=1)
dc_constant_2 = DummyClassifier(strategy="constant", constant=2)
dc_most_frequent = DummyClassifier(strategy="most_frequent")
dc_stratified = DummyClassifier(strategy="stratified")

#using fit to train the classifiers
model_uniform = dc_uniform.fit(train, train_labels)
model_constant_1 = dc_constant_1.fit(train, train_labels)
model_constant_2 = dc_constant_2.fit(train, train_labels)
model_most_frequent = dc_most_frequent.fit(train, train_labels)
model_stratified = dc_stratified.fit(train, train_labels)

#now we make our predictions
preds_uniform = dc_uniform.predict(test)
preds_constant_1 = dc_constant_1.predict(test)
preds_constant_2 = dc_constant_2.predict(test)
preds_most_frequent = dc_most_frequent.predict(test)
preds_stratified = dc_stratified.predict(test)

#print prediction accuracy
accuracy = {}
print("Uniform Classifier: ", accuracy_score(test_labels, preds_uniform))
print("Constant Classifier (1): ", accuracy_score(test_labels, preds_constant_1))
print("Constant Classifier (2): ", accuracy_score(test_labels, preds_constant_2))
print("Most Frequent Classifier: ", accuracy_score(test_labels, preds_most_frequent))
print("Stratified Classifier: ", accuracy_score(test_labels, preds_stratified))

Uniform Classifier:  0.19304347826086957
Constant Classifier (1):  0.1918840579710145
Constant Classifier (2):  0.2
Most Frequent Classifier:  0.1918840579710145
Stratified Classifier:  0.1918840579710145


Εκτυπώνουμε τον confusion matrix. Ο $C$ είναι ο πίνακας για τον οποίο ισχύει οτι $C_{i, j}$ είναι τα δείγματα της κατηγορίας $i$ που ταξινομήθηκαν στην $j$. Όπως φαίνεται οι Uniform και Stratified προβλέπουν οποιαδήποτε κατηγορία, ενώ οι άλλοι 3 είτε επιλέγουν σταθερά μία κατηγορία είτε αυτή με τα περισσότερα δείγματα. Μάλιστα, από τη στιγμή που τα δείγματα στο αρχικό data set είναι ίσα μεταξύ τους, τα πολυπληθέστερα δείγματα στο train set θα είναι τα λιγότερο πολυπληθή στο test set. Άρα ο classifier αυτός θα έχει χειρότερη επίδοση από τους constant classifiers, όπως φαίνεται και από τον confusion matrix.

In [13]:
from sklearn.metrics import confusion_matrix

#produce confusion matrices
cnf_matrix_uniform = confusion_matrix(test_labels, preds_uniform)
cnf_matrix_constant_1 = confusion_matrix(test_labels, preds_constant_1)
cnf_matrix_constant_2 = confusion_matrix(test_labels, preds_constant_2)
cnf_matrix_most_frequent = confusion_matrix(test_labels, preds_most_frequent)
cnf_matrix_stratified = confusion_matrix(test_labels, preds_stratified)

#print confusion matrices
print("Uniform Classifier Confusion Matrix\n")
print(cnf_matrix_uniform, "\n")
print("Constant Classifier (1) Confusion Matrix\n")
print(cnf_matrix_constant_1, "\n")
print("Constant Classifier (2) Confusion Matrix\n")
print(cnf_matrix_constant_2, "\n")
print("Most Frequent Classifier Confusion Matrix\n")
print(cnf_matrix_most_frequent, "\n")
print("Stratified Classifier Confusion Matrix\n")
print(cnf_matrix_stratified, "\n")


Uniform Classifier Confusion Matrix

[[128 130 135 137 132]
 [136 135 147 146 126]
 [136 143 154 146 146]
 [165 142 135 126 135]
 [112 135 144 156 123]] 

Constant Classifier (1) Confusion Matrix

[[662   0   0   0   0]
 [690   0   0   0   0]
 [725   0   0   0   0]
 [703   0   0   0   0]
 [670   0   0   0   0]] 

Constant Classifier (2) Confusion Matrix

[[  0 662   0   0   0]
 [  0 690   0   0   0]
 [  0 725   0   0   0]
 [  0 703   0   0   0]
 [  0 670   0   0   0]] 

Most Frequent Classifier Confusion Matrix

[[662   0   0   0   0]
 [690   0   0   0   0]
 [725   0   0   0   0]
 [703   0   0   0   0]
 [670   0   0   0   0]] 

Stratified Classifier Confusion Matrix

[[131 148 114 134 135]
 [145 138 115 155 137]
 [151 162 142 135 135]
 [145 137 147 118 156]
 [122 138 135 142 133]] 



Εκτύπωση f1-micro average και f1-macro average.

Η ακρίβεια αφορά την ικανότητα του εκτιμητή να εκτιμά ως δείγματα του test set που ανήκουν στη θετική κλάση μόνο αυτά που όντως ανήκουν.<br>
Precision: $$P = \frac{T_p}{T_p+F_p}$$

Η ανάκληση αφορά την ικανότητα του εκτιμητή να εντοπίζει όλα τα δείγματα που ανήκουν στη θετική κλάση.<br>
Recall: $$R = \frac{T_p}{T_p + F_n}$$

F1 score είναι ο αρμονικός μέσος όρος αυτών των δύο.<br>
F1: $$F1 = 2\frac{P \times R}{P+R}$$

Macro average: υπολογίζει f1 ξεχωριστά για κάθε κλάση και παίρνει τον μέσο όρο. Άρα κάθε κλάση αντιμετωπίζεται ισότιμα.

Micro average: ενσωματώνει την πληροφορία για τον αριθμό των δειγμάτων που ανήκουν σε κάθε κλάση, χρησιμοποιώντας τις πραγματικές ποσότητες $T_p$, $F_p$, $F_n$ στον συνολικό υπολογισμό. Άρα είναι προτιμότερη όταν μία κλάση περιέχει αρκετά περισσότερα δείγματα.

In [14]:
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

#f1-micro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='micro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='micro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_2, average='micro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='micro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='micro'), "\n")


#f1-macro
print('\033[1m' + "f1-macro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='macro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='macro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_2, average='macro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='macro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='macro'))



f1-micro
Uniform Classifier:  (0.19304347826086957, 0.19304347826086957, 0.19304347826086957, None)
Constant Classifier (1):  (0.1918840579710145, 0.1918840579710145, 0.1918840579710145, None)
Constant Classifier (2):  (0.2, 0.2, 0.20000000000000004, None)
Most Frequent Classifier:  (0.1918840579710145, 0.1918840579710145, 0.1918840579710145, None)
Stratified Classifier:  (0.1918840579710145, 0.1918840579710145, 0.1918840579710145, None) 

f1-macro
Uniform Classifier:  (0.19291002507750843, 0.19284667886627244, 0.19286849700855277, None)
Constant Classifier (1):  (0.0383768115942029, 0.2, 0.06439688715953307, None)
Constant Classifier (2):  (0.04, 0.2, 0.06666666666666668, None)
Most Frequent Classifier:  (0.0383768115942029, 0.2, 0.06439688715953307, None)
Stratified Classifier:  (0.19213932735878753, 0.19202135812312232, 0.191904151871869, None)


##### Gaussian Naive Bayes

Ο Gaussian Nauve Bayes υποθέτει οτι τα χαρακτηριστικά είναι ανεξάρτητα μεταξύ τους. Έτσι συνδέει κάθε χαρακτηριστικό $x_i$ με κάθε κλάση $y$ με την πιθανότητα $P(x_i \mid y)$, η οποία υποθέτει πως ακολουθεί γκαουσιανή κατανομή. Χρησιμοποιεί τα δεδομένα προκειμένου για κάθε κλάση και χαρακτηριστικό, το οποίο παίρνει συνεχείς τιμές, να βρει τη μέση τιμή $\mu_y$ και τη διακύμανση $\sigma^2_y$ κάθε χαρακτηριστικού για τη κλάση $y$. Στην φάση του testing λαμβάνει υπόψην του τα γινόμενα των παραπάνω πιθανοτήτων των χαρακτηριστικών για κάθε κλάση, καθώς και την πιθανότητα της ίδιας της κλάσης και αναθέτει στο δείγμα την κλάση που μεγιστοποιεί το τελικό γινόμενο.

Παρατηρούμε πως η ενσωμάτωση πληροφορίας, ακόμα και όταν έχουμε κάνει τις παραπάνω υποθέσεις υπερδιπλασιάζει την ακρίβεια των προβλέψεών μας.

Επίσης, όπως φαίνεται από το confusion matrix υπάρχει αναθέτει πολλά δείγματα στην πέμπτη κλάση. Ωστόσο επιτυγχάνει σε μεγάλο βαθμό να εντοπίσει τις επιληπτικές κρίσεις (πρώτη κλάση).

In [15]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

#finding mean values and variations
model_GNB = gnb.fit(train, train_labels)

#making predictions
preds = {}
preds["Gaussian Naive Bayes"] = gnb.predict(test)

#calculate and print prediction accuracy
accuracy = {}
accuracy["Gaussian Naive Bayes"] = accuracy_score(test_labels, preds["Gaussian Naive Bayes"])
print("Gaussian Naive Bayes: ", accuracy["Gaussian Naive Bayes"], "\n")

#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["Gaussian Naive Bayes"])
print('\033[1m' + "Gaussian Naive Bayes - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Gaussian Naive Bayes: ", precision_recall_fscore_support(test_labels, preds["Gaussian Naive Bayes"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("Gaussian Naive Bayes: ", precision_recall_fscore_support(test_labels, preds["Gaussian Naive Bayes"], average='macro'))



Gaussian Naive Bayes:  0.4260869565217391 

Gaussian Naive Bayes - Confusion Matrix
[[540 117   1   4   0]
 [ 37  95 100  92 366]
 [  1  72 128 132 392]
 [  1 139 148 202 213]
 [  0  18  87  60 505]] 

f1-micro
Gaussian Naive Bayes:  (0.4260869565217391, 0.4260869565217391, 0.4260869565217391, None)
f1-macro
Gaussian Naive Bayes:  (0.4356619753028668, 0.4342028336361641, 0.41257019149951085, None)


##### kNN

Ο kNN υπολογίζει, για κάθε δείγμα του test set, τους k κοντινότερους γείτονές του, οι οποίοι είναι δείγματα του train set, στον n-διάστατο χώρο διαστάσεων των χαρακτηριστικών εισόδου. Αποφασίζει την κλάση του νέου δείγματα παίρνοντας είτε την πλειοψηφία των γειτόνων είτε λαμβάνοντας υπόψην και τις αποστάσεις του από αυτούς. Ως συνάρτηση απόστασης χρησιμοποιείται κυρίως η ευκλείδια. <br>
Εξαιρετικά σημαντική για την απόδοσή του είναι η υπερπαράμετρος k. 

Το γεγονός οτι πρέπει να συγκρίνουμε την απόσταση κάθε νέου δείγματος στο train set με το νέο δείγμα καθιστά τη διαδικασία πρόβλεψης πολύ χρονοβόρα.
Ωστόσο, η μεγάλη διαφορά από τον GNB είναι οτι δεν κάνει υποθέσεις για τις εξαρτήσεις των χαρακτηριστικών και τις κατανομές των πιθανοτήτων.

Παρατηρούμε οτι επιτυγχάνει καλύτερες προβλέψεις από τον kNN, ωστόσο το ποσοστό είναι ακόμα χαμηλό.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) #setting k to 5

#saves training samples and their labels
knn.fit(train, train_labels)

#making predictions
preds["kNN"] = knn.predict(test)

#calculate and print prediction accuracy
accuracy["kNN"] = accuracy_score(test_labels, preds["kNN"])
print("kNN: ", accuracy["kNN"])

kNN:  0.4707246376811594


In [17]:
#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["kNN"])
print('\033[1m' + "kNN - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("kNN: ", precision_recall_fscore_support(test_labels, preds["kNN"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("kNN: ", precision_recall_fscore_support(test_labels, preds["kNN"], average='macro'))

kNN - Confusion Matrix
[[447 113  77  22   3]
 [  2 465 214   0   9]
 [  0 265 452   1   7]
 [  0 156 208 203 136]
 [  0 274 335   4  57]] 

f1-micro
kNN:  (0.4707246376811594, 0.4707246376811594, 0.4707246376811594, None)
f1-macro
kNN:  (0.5727557190931136, 0.4692849957931807, 0.45847596510687466, None)


##### Multi Layer Perceptron

Ένα MLP περιέχει το input layer, το οποίο δέχεται τις εισόδους, 1 ή περισσότερα ενδιάμεσα επίπεδα, τα οποία αποτελούνται από νευρώνες που δέχονται εισόδους από τους νευρώνες του προηγούμενου επιπέδου, εφαρμόζουν βάρη σε αυτές και τις προσθέτουν μαζί με ένα bias. Μετά εφαρμόζουν στο αποτέλεσμα μία μη γραμμική συνάρτηση δημιουργώντας έτσι την έξοδο που προωθείται στο επόμενο επίπεδο. Στο τέλος έχουν ένα output layer, το οποίο κάνει και την τελική απόφαση.

Κατά την εκπαίδευσή τους, ελαχιστοποιούν ένα κριτήριο αλλάζοντας τις τιμές των βαρών και των biases τους. 

Παρατηρούμε οτι δεν αποδίδει καλύτερα ούτε από τους Dummy Classifiers.

In [18]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,), random_state=1)

#training
clf.fit(train, train_labels)

#predicting
preds["Multi Layer Perceptron"] = clf.predict(test)

#calculate and print prediction accuracy
accuracy["Multi Layer Perceptron"] = accuracy_score(test_labels, preds["Multi Layer Perceptron"])
print("Multi Layer Perceptron: ", accuracy["Multi Layer Perceptron"])

#produce and print confusion matrix
cnf_matrix_GNB = confusion_matrix(test_labels, preds["Multi Layer Perceptron"])
print('\033[1m' + "Multi Layer Perceptron - Confusion Matrix" + '\033[0m')
print(cnf_matrix_GNB, "\n")

#f1-micro/macro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Multi Layer Perceptron: ", precision_recall_fscore_support(test_labels, preds["Multi Layer Perceptron"], average='micro'))
print('\033[1m' + "f1-macro" + '\033[0m')
print("Multi Layer Perceptron: ", precision_recall_fscore_support(test_labels, preds["Multi Layer Perceptron"], average='macro'))

Multi Layer Perceptron:  0.18985507246376812
Multi Layer Perceptron - Confusion Matrix
[[213 248  30 131  40]
 [222 287   4 160  17]
 [190 314   6 195  20]
 [182 341  27 121  32]
 [211 283  13 135  28]] 

f1-micro
Multi Layer Perceptron:  (0.18985507246376812, 0.18985507246376812, 0.18985507246376812, None)
f1-macro
Multi Layer Perceptron:  (0.16930531834311024, 0.19197613792011617, 0.1541434580968996, None)


# Δ. Βελτιστοποίηση Ταξινομητών

Χρησιμοποιούμε την διαδικασία pipeline για την εφαρμογή διαδοχικών δεν υπάρχει μεγάλη ανάγκη για εφαρμογή oversampling ή undersampling. Αυτό επιβεβαιώθηκε όταν κάναμε τις ανάλογες δοκιμές. 

* Λόγω του ισορροπημένου dataset, όπως φαίνεται και παρακάτω, δεν χρειάζεται να χρησιμοποιήσουμε oversampler ή undersampler.

* Διαστατικότητα: Χρησιμοποιούμε τον μετασχηματιστή VarianceThreshold, ο οποίος μειώνει τον αριθμό των χαρακτηριστικών με βάση την διακύμανση των τιμών του στα δείγματα (επιλογή χαρακτηριστικών). Όταν η διακύμανση είναι μικρή θεωρούμε οτι το χαρακτηριστικό δεν προσφέρει πολλή πληροφορία για την κατηγοριοποίηση. <br>
Επίσης χρησιμοποιούμε τον PCA για την ανάλυση των δεδομένων σε κύριες συνιστώσες και την χρήση των συνιστωσών με την περισσότερη διακύμανση (εξαγωγή χαρακτηριστικών), δηλαδή πληροφορία. 

* Κανονικοποίηση: Αμβλύνουμε τις διαφορές μεταξύ των τιμών των χαρακτηριστικών. Αν ένα χαρακτηριστικό έχει πολύ μεγαλύτερες τιμές από ένα άλλο η σημασία του σε εκτιμητές όπως ο kNN, που μετρά τις αποστάσεις από τα χαρακτηριστικά, είναι μεγαλύτερη χωρίς αυτό να σημαίνει οτι παρέχει περισσότερη πληροφορία για την κατηγοριοποίηση. Χρησιμοποιούμε δύο μετασχηματιστές κανονικοποίησης, τον scaler και τον min_max_scaler. 

Η σειρά που ακολουθείται είναι η εξής:
0. minmax αν VarianceThreshold για να μην επηρεαστεί η επιλογή από τις τιμές των χαρακτηριστικών
1. Κανονικοποίηση (minmax ή z-score)
2. PCA
3. Εκτιμητής



Χρήση grid search για την βελτιστοποίηση των υπερπαραμέτρων. Φτιάχνουμε σύνολο πιθανών συνδυασμών τιμών των παραμέτρων για να βρόυμε τον βέλτιστο. Υπολογίζεται ο μέσος όρος σε όλα τα folds (5 εδώ) του cross-validation με βάση της f1 μετρικές. 

In [19]:
print("train frequencies:", np.bincount(train_labels)[1:])

train frequencies: [1638 1610 1575 1597 1630]


Εκτυπώνουμε την διακύμανση αφού έχουμε εφαρμόσει minmax για να υπολογίσουμε σωστά τα thresholds.

In [20]:
from sklearn.preprocessing import MinMaxScaler 

min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(train)
train_variance = X_train_minmax.var(axis=0)
print(np.max(train_variance))
print(np.min(train_variance))


0.003836979589388692
0.0016974625748715367


In [0]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
import time

In [0]:
"""

"""
def GridSearch(data, preprocessing_steps, parameters, classifier, Scoring):
  (train, train_labels, test, test_labels) = data
  dictionary = {} #dictionary for Gridsearch grid

  #checking for sampling condition
  if parameters["sampling_strategy"] < 0 or parameters["sampling_strategy"] > 1:
    parameters["sampling_strategy"] = 0
  if preprocessing_steps["sampling"] == "Over":
    ros = RandomOverSampler(parameters["sampling_strategy"])
  elif preprocessing_steps["sampling"] == "Under":
    ros = RandomUnderSampler(parameters["sampling_strategy"])
  else:
    ros = None

  #checking for Variance Threshold selector
  if preprocessing_steps["selector"] == "VT":
    preprocessing_steps["scaler"] == None   #no need for a scaler since we will use minmax in the beginning
    scaler_minmax = MinMaxScaler()   #minmax is applied before VT
    selector = VarianceThreshold()
    dictionary["selector__threshold"] = parameters["vthreshold"]
  else:
    scaler_minmax = None
    selector = None

  #checking for the use of a scaler
  if preprocessing_steps["scaler"] == "minmax":
    scaler = MinMaxScaler()
  elif preprocessing_steps["scaler"] == "zscore":
    scaler = StandardScaler()
  else:
    scaler = None

  #checking for the use of PCA
  if preprocessing_steps["extractor"] == "PCA":
    pca = PCA()
    dictionary["pca__n_components"] = parameters["n_components"]
  else:
    pca = None
  
  #creating estimator
  if classifier == "GNB":
    clf = GaussianNB()
  elif classifier == "kNN":
    clf = KNeighborsClassifier()
    dictionary["kNN__n_neighbors"] = parameters["k"]
    dictionary["kNN__weights"] = parameters["weights"]
    dictionary["kNN__metric"] = parameters["metrics"]
  elif classifier == "MLP":
    clf = MLPClassifier(random_state=1, learning_rate_init = 0.05)
    dictionary["MLP__solver"] = parameters["solver"]
    dictionary["MLP__max_iter"] = parameters["max_iter"]
    dictionary["MLP__alpha"] = parameters["alpha"]
    dictionary["MLP__hidden_layer_sizes"] = parameters["hidden_layer_sizes"]
    dictionary["MLP__activation"] = parameters["activation"]
    dictionary["MLP__learning_rate"] = parameters["learning_rate"]


  #create pipeline, using memory so that transformed data is saved and not recomputed with each fold change
  pipe = Pipeline(steps=[('sampler', ros), ('minmax_scaler', scaler_minmax), ('selector', selector), ('scaler', scaler), ('pca', pca), (classifier, clf)], memory = 'tmp')  

  #create estimator
  estimator = GridSearchCV(pipe, dictionary, cv=5, scoring=Scoring, n_jobs=-1) #number of folds is 5

  #fit and predict
  start_time = time.time()
  estimator.fit(train, train_labels)
  preds = estimator.predict(test)
  t = time.time() - start_time
  print("Συνολικός χρόνος fit και predict: %s seconds" % t)
  accuracy = accuracy_score(test_labels, preds)
  print("Accuracy: ", accuracy)
  print("Precision_Recall_fscore: ", precision_recall_fscore_support(test_labels, preds, average='micro'))
  print("Precision_Recall_fscore: ", precision_recall_fscore_support(test_labels, preds, average='macro'))

  #print(estimator.best_estimator_)
  print(estimator.best_params_, "\n")

  return preds, t, precision_recall_fscore_support(test_labels, preds, average='micro')[2], precision_recall_fscore_support(test_labels, preds, average='macro')[2]

##### Gaussian Naive Bayes

In [23]:
classifier = "GNB"

#setting up parameters
data = (train, train_labels, test, test_labels)
vthreshold = [0, 0.0017, 0.002, 0.0023] 
n_components = [10, 20, 30, 40, 50, 60, 70]

preprocessing_steps = {"sampling" : None, "selector" : None, "scaler" : None, "extractor" : None}
parameters = {"sampling_strategy" : 0, "vthreshold" : vthreshold, "n_components" : n_components}

#f1-micro
best_GNB_micro = 0
for selector in (None, "VT"):
  for scaler in (None, "minmax", "zscore"):
    for extractor in (None, "PCA"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-micro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_micro')
      preds, t, f1_micro, f1_macro = results
      if f1_micro > best_GNB_micro:
        print("Best so far:", f1_micro, "\n")
        best_GNB_micro_time = t
        best_GNB_micro_preds = preds
        best_GNB_micro = f1_micro


#f1-macro
best_GNB_macro = 0
for selector in (None, "VT"):
  for scaler in (None, "minmax", "zscore"):
    for extractor in (None, "PCA"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-macro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_macro')
      preds, t, f1_micro, f1_macro = results
      if f1_macro > best_GNB_macro:
        print("Best so far:", f1_macro, "\n")
        best_GNB_macro_time = t
        best_GNB_macro_preds = preds
        best_GNB_macro = f1_macro


 f1-micro Preprocessing steps:  {'sampling': None, 'selector': None, 'scaler': None, 'extractor': None} 
Συνολικός χρόνος fit και predict: 1.7714924812316895 seconds
Accuracy:  0.4260869565217391
Precision_Recall_fscore:  (0.4260869565217391, 0.4260869565217391, 0.4260869565217391, None)
Precision_Recall_fscore:  (0.4356619753028668, 0.4342028336361641, 0.41257019149951085, None)
{} 

Best so far: 0.4260869565217391 

 f1-micro Preprocessing steps:  {'sampling': None, 'selector': None, 'scaler': None, 'extractor': 'PCA'} 


KeyboardInterrupt: ignored

##### kNN

Η αποτελεσματικότητα του kNN, εκτός από το k, εξαρτάται και από την ύπαρξη βαρών στον υπολογισμό των γειτόνων, δηλαδή από το αν οι κοντινοί γείτονες αποκτούν μεγαλύτερη σημασία από τους μακρινότερους. Εφαρμόζουμε 2 περιπτώσεις:
* uniform: όλοι οι γείτονες έχουν την ίδια σημασία
* distance: η σημασία είναι αντιστρόφως ανάλογη της απόστασης

Εκτός αυτού σημασία έχει και ο τρόπος υπολογισμού της απόστασης. Εφαρμόζουμε 2 τρόπους υπολογισμού:
* manhattan
* euclidean

In [0]:
classifier = "kNN"

#setting up parameters
data = (train, train_labels, test, test_labels)
vthreshold = [0, 0.0017, 0.002, 0.0023] 
n_components = [10, 15, 20, 25, 30, 35]
k = [1, 2, 3]
weights = ["uniform", "distance"]
metrics = ["euclidean", "manhattan"]

preprocessing_steps = {"sampling" : None, "selector" : None, "scaler" : None, "extractor" : None}
parameters = {"sampling_strategy" : 0, "vthreshold" : vthreshold, "n_components" : n_components, "k" : k, "weights" : weights, "metrics" : metrics}

#f1-micro
best_kNN_micro = 0
for selector in (None, "VT"):
  for scaler in (None, "minmax", "zscore"):
    for extractor in (None, "PCA"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-micro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_micro')
      preds, t, f1_micro, f1_macro = results
      if f1_micro > best_kNN_micro:
        print("Best so far:", f1_micro, "\n")
        best_kNN_micro_time = t
        best_kNN_micro_preds = preds
        best_kNN_micro = f1_micro

#f1-macro
best_kNN_macro = 0
for selector in (None, "VT"):
  for scaler in (None, "minmax", "zscore"):
    for extractor in (None, "PCA"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-macro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_macro')
      preds, t, f1_micro, f1_macro = results
      if f1_macro > best_kNN_macro:
        print("Best so far:", f1_micro, "\n")
        best_kNN_macro_time = t
        best_kNN_macro_preds = preds
        best_kNN_macro = f1_macro

##### Multi Layer Perceptron

Οι παράμετροι που εξετάζουμε είναι οι παρακάτω:
* Ο αριθμός των νευρώνων στον κρυμμένο επίπεδο.
* Η μη γραμμική συνάρτηση που εφαρμόζεται. Μπορεί να εφαρμοστεί η σιγμοειδής, η υπερβολική εφαπτομένη ή η relu.
* Ο τρόπος ενημέρωσης των βαρών. Εφαρμόσαμε μία quasi-Newton method και την stochastic gradient descent.
* alpha: Η παράμετρος για την F2 κανονικοποίηση, η οποία βοηθά στην αποφυγή του overfitting, φροντίζοντας τα βάρη να μη μαθαίνουν τις ιδιομορφίες του dataset.
* Μέγιστος αριθμός επαναλήψεων που επιτρέπουμε στον αλγόρθμο.
* Το learing rate, το οποίο αφορά την "ώθηση" που δίνουμε σε κάθε ενημέρωση των παραμέτρων. Μπορεί να είναι σταθερό καθόλη τη διάρκεια της εκπαίδευσης ή να μεταβάλλεται.

In [0]:
classifier = "MLP"

#setting up parameters
data = (train.iloc[1:1000, :], train_labels.iloc[1:1000], test, test_labels)
vthreshold = [0.0017, 0.002, 0.0023] 
n_components = [30, 35, 40]

hidden_layer_sizes = [(5, ), (10, ), (15, )]
activation = ["tanh", "relu"]
solver = ["lbfgs", "sgd"]
max_iter = [40, 80, 120]
alpha = [0.00001, 0.0001, 0.001]
learning_rate = ["constant", "invscaling"]

preprocessing_steps = {"sampling" : None, "selector" : None, "scaler" : None, "extractor" : None}
parameters = {"sampling_strategy" : 0, "vthreshold" : vthreshold, "n_components" : n_components, "solver": solver, "max_iter": max_iter, "alpha": alpha, "hidden_layer_sizes": hidden_layer_sizes, "activation": activation, "learning_rate" : learning_rate}

#f1-micro
best_MLP_micro = 0
for scaler in (None, "minmax", "zscore"):
  for extractor in (None, "PCA"):
    for selector in (None, "VT"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-micro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_micro')
      preds, t, f1_micro, f1_macro = results
      if f1_micro > best_MLP_micro:
        print("Best so far:", f1_micro, "\n")
        best_MLP_micro_time = t
        best_MLP_micro_preds = preds
        best_MLP_micro = f1_micro

#f1-macro
best_MLP_macro = 0
for scaler in (None, "minmax", "zscore"):
  for extractor in (None, "PCA"):
    for selector in (None, "VT"):
      preprocessing_steps["selector"] = selector
      preprocessing_steps["scaler"] = scaler
      preprocessing_steps["extractor"] = extractor
      print('\033[1m', "f1-macro", "Preprocessing steps: ", preprocessing_steps, '\033[0m')
      results = GridSearch(data, preprocessing_steps, parameters, classifier, Scoring = 'f1_macro')
      preds, t, f1_micro, f1_macro = results
      if f1_macro > best_MLP_macro:
        print("Best so far:", f1_macro, "\n")
        best_MLP_macro_time = t
        best_MLP_macro_preds = preds
        best_MLP_macro = f1_macro

 f1-micro Preprocessing steps:  {'sampling': None, 'selector': None, 'scaler': None, 'extractor': None} 
Συνολικός χρόνος fit και predict: 158.21877598762512 seconds
Accuracy:  0.3310144927536232
Precision_Recall_fscore:  (0.3310144927536232, 0.3310144927536232, 0.3310144927536232, None)
Precision_Recall_fscore:  (0.3276732863394307, 0.33167211437304855, 0.31636655037476535, None)
{'MLP__activation': 'relu', 'MLP__alpha': 0.0001, 'MLP__hidden_layer_sizes': (15,), 'MLP__learning_rate': 'invscaling', 'MLP__max_iter': 80, 'MLP__solver': 'sgd'} 

Best so far: 0.3310144927536232 

 f1-micro Preprocessing steps:  {'sampling': None, 'selector': 'VT', 'scaler': None, 'extractor': None} 
Συνολικός χρόνος fit και predict: 433.99163484573364 seconds
Accuracy:  0.38666666666666666
Precision_Recall_fscore:  (0.38666666666666666, 0.38666666666666666, 0.38666666666666666, None)
Precision_Recall_fscore:  (0.4006256716191846, 0.39398396696614435, 0.36029309376794955, None)
{'MLP__activation': 'tanh', '